In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import polars as pl
import re

data_path = '/media/chris/4T1/bigFileRepo/physionet.datasets/mimiciv/3.0/hosp/'
DAzero_path = '~/Documents/mimic/DAzero/'

In [2]:
# functions

def format_ndc_codes(df, column_name='ndc', new_column_name='new_ndc'):
    def format_single_ndc(ndc):
        # Convert scientific notation to integer string
        ndc_str = f'{ndc:.0f}'
        
        # Pad with leading zeros to ensure 11 digits
        ndc_padded = ndc_str.zfill(11)
        
        # Remove the last two digits
        return ndc_padded[:-2]
    
    # Apply the formatting function to the specified column
    df[new_column_name] = df[column_name].apply(format_single_ndc)
    
    return df

def format_ndc_codes_hyphenated(df, column_name='productndc', new_column_name='new_ndc'):
    def format_single_ndc(ndc):
        # Split the NDC code into parts
        parts = ndc.split('-')
        
        if len(parts) != 2:
            return ndc  # Return original if not in expected format
        
        # Pad each part with leading zeros
        part1 = parts[0].zfill(5)
        part2 = parts[1].zfill(4)
        
        # Combine parts without hyphen
        return part1 + part2
    
    # Apply the formatting function to the specified column
    df[new_column_name] = df[column_name].apply(format_single_ndc)
    
    return df

def extract_pharm_class(df, column_name='pharm_classes'):
    def extract_text(text):
        if pd.isna(text):
            return pd.NA
        
        # Use regex to find the text before "[EPC]"
        match = re.search(r'(?:^|,)\s*([^,]+?)\s*\[EPC\]', text)
        
        if match:
            # Strip leading/trailing whitespace
            return match.group(1).strip()
        else:
            # If [EPC] is not found, return NA
            return pd.NA

    # Apply the extraction function to the specified column
    df['extracted_class'] = df[column_name].apply(extract_text)
    
    return df

In [3]:
# load the mimin iv prescrtiptions data

prescriptions = pl.read_csv(
    data_path + 'prescriptions.csv',
    ignore_errors=True
    #n_rows=10000  # Limit to 10,000 rows, adjust as needed
)

#print(prescriptions.columns)

meds = prescriptions.to_pandas()

In [4]:
# manage the incorrectly formatted NDC codes in the prescription dataframe

meds_formatted = format_ndc_codes(meds)
#print(meds_formatted.head(1))

In [5]:
# load ndc product table (downloaded from FDA website)
ndc = pd.read_csv(DAzero_path + 'data/NDC_product_table.csv', encoding='latin1')
# convert column names to lowercase
ndc.columns = ndc.columns.str.lower()
print(ndc.columns)

Index(['productid', 'productndc', 'producttypename', 'proprietaryname',
       'proprietarynamesuffix', 'nonproprietaryname', 'dosageformname',
       'routename', 'startmarketingdate', 'endmarketingdate',
       'marketingcategoryname', 'applicationnumber', 'labelername',
       'substancename', 'active_numerator_strength', 'active_ingred_unit',
       'pharm_classes', 'deaschedule', 'ndc_exclude_flag',
       'listing_record_certified_through'],
      dtype='object')


In [6]:
# ndc codes also incorrectly formatted in the NDC product table
# correct the codes, to match the prescription dataset

ndc = format_ndc_codes_hyphenated(ndc)
# print(meds_hyphenated)

In [7]:
# extract the correct info on the drug class from the pharm_classes column
ndc = extract_pharm_class(ndc)
# print(ndc.head(1))

In [8]:
# now merge meds and ndc
# around 6M rows lost in this step - I think due to old NDC codes not being in the NDC product table (which comes from the FDA)
# I have some old NDC code lists we could use, but hopefully we don't need to do this
print(meds.shape)
meds_merged = pd.merge(meds_formatted, ndc, how='inner', left_on='new_ndc', right_on='new_ndc')
print(meds_merged.shape)

(20292611, 22)
(14084704, 43)


In [9]:
# show the extracted classes
print(meds_merged['extracted_class'].unique())

['Loop Diuretic' 'Anticholinergic'
 'Human Immunodeficiency Virus Integrase Strand Transfer Inhibitor'
 'Aldosterone Antagonist' 'beta2-Adrenergic Agonist'
 'Human Immunodeficiency Virus Nucleoside Analog Reverse Transcriptase Inhibitor'
 'Anti-coagulant' <NA> 'Rifamycin Antibacterial' 'Osmotic Laxative'
 'Sulfonamide Antimicrobial' 'Corticosteroid' 'Opioid Agonist'
 'HMG-CoA Reductase Inhibitor' 'Cholinesterase Inhibitor'
 'Atypical Antipsychotic' 'Nonergot Dopamine Agonist'
 'Monoamine Oxidase Inhibitor' 'Cephalosporin Antibacterial'
 'Serotonin-3 Receptor Antagonist' 'Low Molecular Weight Heparin'
 'Dopamine-2 Receptor Antagonist' 'Phenothiazine'
 'Nonsteroidal Anti-inflammatory Drug' 'Proton Pump Inhibitor'
 'Glycopeptide Antibacterial' 'beta-Adrenergic Blocker'
 'Angiotensin Converting Enzyme Inhibitor' 'Bisphosphonate'
 'Calculi Dissolution Agent' 'Nitroimidazole Antimicrobial'
 'Prostaglandin Analog' 'Benzodiazepine' 'Quinolone Antimicrobial'
 'Oxytocic' 'Penicillin-class Antiba

In [10]:
# create list of extracted classes unique values
extracted_classes = meds_merged['extracted_class'].unique()
# remove nan
extracted_classes = extracted_classes[~pd.isna(extracted_classes)]
print(extracted_classes)

# create a dictionary of extracted classes
class_dict = dict(zip(extracted_classes, range(len(extracted_classes))))
print(class_dict)

# save out extracted classes
class_df = pd.DataFrame(extracted_classes, columns=['class'])
class_df.to_csv(DAzero_path + 'data/extracted_classes.csv', index=False)

['Loop Diuretic' 'Anticholinergic'
 'Human Immunodeficiency Virus Integrase Strand Transfer Inhibitor'
 'Aldosterone Antagonist' 'beta2-Adrenergic Agonist'
 'Human Immunodeficiency Virus Nucleoside Analog Reverse Transcriptase Inhibitor'
 'Anti-coagulant' 'Rifamycin Antibacterial' 'Osmotic Laxative'
 'Sulfonamide Antimicrobial' 'Corticosteroid' 'Opioid Agonist'
 'HMG-CoA Reductase Inhibitor' 'Cholinesterase Inhibitor'
 'Atypical Antipsychotic' 'Nonergot Dopamine Agonist'
 'Monoamine Oxidase Inhibitor' 'Cephalosporin Antibacterial'
 'Serotonin-3 Receptor Antagonist' 'Low Molecular Weight Heparin'
 'Dopamine-2 Receptor Antagonist' 'Phenothiazine'
 'Nonsteroidal Anti-inflammatory Drug' 'Proton Pump Inhibitor'
 'Glycopeptide Antibacterial' 'beta-Adrenergic Blocker'
 'Angiotensin Converting Enzyme Inhibitor' 'Bisphosphonate'
 'Calculi Dissolution Agent' 'Nitroimidazole Antimicrobial'
 'Prostaglandin Analog' 'Benzodiazepine' 'Quinolone Antimicrobial'
 'Oxytocic' 'Penicillin-class Antibacteri

In [11]:
# classes of interest
# NB corticosteroids only have a single class
antibacterial = meds_merged[meds_merged['extracted_class'].str.contains('Antibacterial', na=False)]['extracted_class'].unique()
antimicrobial = meds_merged[meds_merged['extracted_class'].str.contains('Antimicrobial', na=False)]['extracted_class'].unique()
antifungal = meds_merged[meds_merged['extracted_class'].str.contains('Antifungal', na=False)]['extracted_class'].unique()

In [12]:
print(antibacterial)
print(antimicrobial)
print(antifungal)

['Rifamycin Antibacterial' 'Cephalosporin Antibacterial'
 'Glycopeptide Antibacterial' 'Penicillin-class Antibacterial'
 'Aminoglycoside Antibacterial' 'Penem Antibacterial'
 'Oxazolidinone Antibacterial' 'Lincosamide Antibacterial'
 'Lipopeptide Antibacterial' 'Tetracycline-class Antibacterial'
 'RNA Synthetase Inhibitor Antibacterial' 'Nitrofuran Antibacterial'
 'Monobactam Antibacterial' 'Polymyxin-class Antibacterial'
 'Amphenicol-class Antibacterial' 'Sulfonamide Antibacterial'
 'Macrolide Antibacterial' 'Lipoglycopeptide Antibacterial'
 'Dihydrofolate Reductase Inhibitor Antibacterial'
 'Methylated Sulfonamide Antibacterial']
['Sulfonamide Antimicrobial' 'Nitroimidazole Antimicrobial'
 'Quinolone Antimicrobial' 'Macrolide Antimicrobial'
 'Tetracycline-class Antimicrobial' 'Polyene Antimicrobial']
['Azole Antifungal' 'Polyene Antifungal' 'Echinocandin Antifungal'
 'Lipid-based Polyene Antifungal' 'Nucleoside Analog Antifungal'
 'Allylamine Antifungal']


In [13]:
# reset the meds_merged index
meds_merged = meds_merged.reset_index(drop=True)

# create a smaller dataframe with only the relevant columns
meds_extract = meds_merged[['subject_id', 'hadm_id', 'starttime', 'drug', 'prod_strength',
       'dose_val_rx', 'dose_unit_rx',
       'doses_per_24_hrs', 'extracted_class']]

print(meds_extract['extracted_class'].value_counts())

extracted_class
Anti-coagulant                                1585020
Osmotic Laxative                              1570737
Opioid Agonist                                1081605
Potassium Salt                                 611669
Insulin Analog                                 571405
                                               ...   
Progestin Antagonist                                1
Cholinesterase Reactivator                          1
Photoactivated Radical Generator                    1
Thyroid Stimulating Hormone                         1
B Lymphocyte Stimulator-specific Inhibitor          1
Name: count, Length: 334, dtype: int64


In [14]:
# save out the extracted data
meds_extract.to_csv(DAzero_path + 'data/mimic_prescriptions_extracted.csv', index=False)